In [1]:
##Load metadata from file

import ast

d = {}
nary = {}
with open("metadata_trial4.txt", 'r') as f:
    for line in f:
        d = (ast.literal_eval(line))
        nary.update({d['id'] : d})

In [2]:
#Filter terms that have children and aspect

filnary = {}
kilnary = {}
nochild = {}

for key, value in nary.items():
    if value.get('children') != None:
        filnary = ({'id' : value['id'], 'children' : value['children']})
        kilnary.update({key : filnary})
    else:
        filnary = ({'id' : value['id'],})
        nochild.update({key : filnary})

In [3]:
#get parents

for key, value in kilnary.items():
    for item in value['children']:
        if nary[item['id']].get('parent') == None:
            nary[item['id']]['parent'] = []
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
        else:
            nary[item['id']]['parent'].append({'id' : key, 'relation' : item['relation']})
            
for key, value in nary.items():
    if value.get('parent') == None:
        value['parent'] = []

In [4]:
#empty children class for nochilds

for key, value in nary.items():
    if value.get('children') == None:
        value['children'] = []

In [5]:
#get rid of the two motherfuckers

for k,v in nary.items():
    for item in v['parent']:
        for item2 in v['children']:
            if item == item2:
                del(v['parent'][v['parent'].index(item)])
                del(v['children'][v['children'].index(item)])

In [6]:
#hashmap

hashmap = {}

for key, value in nary.items():
    hashmap[key] = False

In [7]:
#import owlready2

import types
from owlready2 import *
onto = get_ontology("http://test.org/onto.owl")

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [10]:
#define core classes

with onto:
    class therapeutic_agent(Thing):
        namespace = onto
    class biological_process_main(Thing):
        namespace = onto
    class molecular_function_main(Thing):
        namespace = onto
    class drug(Thing):
        namespace = onto

In [11]:
def recursive_classmaker_down(term):
    for item in nary[term]['children']:
        if item['relation'] == 'is_a' and hashmap[item['id']] == False:
            with onto:
                nc = types.new_class(item['id'][3:], (getattr(onto, term[3:]),))
            recursive_classmaker_down(item['id'])
            hashmap[item['id']] = True
    return 0

In [12]:
#Using classmaker to make FTC classes

for key, value in nary.items():
    if value['parent'] == [] and value['aspect'] == 'biological_process':
        with onto:
            nc = types.new_class(key[3:], (biological_process_main,))
        if hashmap[key] == False:
            recursive_classmaker_down(key)
    if value['parent'] == [] and value['aspect'] == 'molecular_function':
        with onto:
            nc = types.new_class(key[3:], (molecular_function_main,))
        if hashmap[key] == False:
            recursive_classmaker_down(key)
   

In [37]:
#delete cellular components fron nary
#remove anything that is not a biological_process or molecular_function

c = 0
keys = []
for key, value in nary.items():
    if value['aspect'] != 'biological_process' and value['aspect'] != 'molecular_function':
        keys.append(key)
        c+=1
print(c)
for i in keys:
    del nary[i]

4367


In [39]:
classes_not_created

['GO:0150136',
 'GO:0150138',
 'GO:0150139',
 'GO:0150151',
 'GO:0150152',
 'GO:0150153',
 'GO:0098504',
 'GO:1902496',
 'GO:0061396',
 'GO:0150161',
 'GO:0150162',
 'GO:0150160']

In [46]:
#Handling poor discount classes

#list of classes created

GO_classes_created = [('GO:' + str(x)[5:]) for x in list(onto.classes())]


classes_not_created = []

for k, v in nary.items():
    if k not in GO_classes_created:
        classes_not_created.append(k)


l = list(onto.classes())


In [54]:
for x in classes_not_created:
    for a in nary[x]['parent']:
        if a['id'][3:] in GO_classes_created:
            with onto:
                nc = types.new_class(x[3:], (getattr(onto, a['id'][3:]),))
            recursive_classmaker_down(x)

In [55]:
len(list(onto.classes()))

42985

In [53]:
len(classes_not_created)

12

In [58]:
nary['GO:0000139']

KeyError: 'GO:0000139'

In [47]:
for i in l:
    if ('GO:' + str(i)[5:]) in nary:
        for item in nary['GO:' + str(i)[5:]]['children']:
            for ancestor in list(i.ancestors()):
                if item['id'] == str('GO:' + str(ancestor)[5:]):
                    print(item['id'])

In [62]:
for k,v in nary.items():
    for item in v['parent']:
        for item2 in v['children']:
            if item == item2:
                print(item)

In [56]:
onto.save(file = "onto", format = "rdfxml")

In [ ]:
nary